# <p class='alert alert-info'>Proyecto final</p>

## <p class='alert alert-warning'>Introducción: </p>

<p>En ester proyecto final se tratará de resolver la problemática de encontrar localizar una ubicación adecuada para construir un nuevo hotel. Como en el documento informativo se explica, la inteción es poder crear ubicaciones semejantes a nuestra ubicación deseada. <p>

## <p class='alert alert-info'>Importación de librerías</p>

In [1]:
###
# Importaciones
###
# Data
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
# Request
import requests
# Scrapper
from bs4 import BeautifulSoup
# Mapas
import geocoder
from geopy.geocoders import Nominatim
import folium
# POST - GET request
import requests # librería para manejar solicitudes
# Machine Learning

## <p class='alert alert-info'>Obtención de datos</p>

## <p class='alert alert-info'>DataFrame resultante</p>

## <p class='alert alert-info'>DataFrame resultante</p>

## <p class='alert alert-info'>Mapas de Toronto</p>